In [1]:
!pip install langchain

In [ ]:
# Important, mettre dans un terminal : 
# ollama pull llama2
# ollama run dolphin-phi

In [2]:
from langchain_community.llms import Ollama

# llm = Ollama(model="llama2")
llm = Ollama(model="dolphin-phi")

In [2]:
llm.invoke("What are the basicis with git for coding projects ?")

"1. **Initial Installation:** Git is first installed on your system. It can be downloaded from many different sources like https://git-scm.com/downloads/. Once installed, you can install it using a tool called `curl` or by simply copying and pasting the URL in your terminal.\n\n2. **Creating a Repository:** Create a new repository for your coding project. This can be done from the command line by running `git init`.\n\n3. **Commit Code Changes:** Whenever you make changes to your code, you need to commit them to the repository. You can do this using the `git add` and `git commit` commands.\n\n4. **Checkout a Version of the Repository:** If you want to work on a specific version of the project, you can checkout that version from the repository's history. This can be done using the `git checkout` command.\n\n5. **Lists Files and Directories in a Project:** To see what files and directories are in a project, use the `git ls-files` command.\n\n6. **View Changes Between Versions:** If you w

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

chain = prompt | llm 

In [4]:
chain.invoke({"input": "how can langsmith help with testing?"})

"As a language processing software (LangSmith), I can assist with various aspects of testing in a few different ways:\n\n1. Code Review and Refactoring: LangSmith can analyze your code for readability, style adherence, and adherence to coding standards. This helps you improve the quality of your codebase over time, making it easier to understand, maintain, and test.\n\n2. Find and Fix Errors: I can identify potential issues in your code such as syntax errors, logical flaws, and possible bugs. This reduces the manual work required for testing and increases the reliability of your software.\n\n3. Testing-Friendly Language Style: By providing a consistent and well-structured writing style, LangSmith ensures that your documentation is easy to read and understand. This helps developers better comprehend the requirements and design, ultimately improving the effectiveness of their testing.\n\n4. Automated Documentation Generation: With the help of AI and machine learning algorithms, I can gen

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = prompt | llm | output_parser

In [8]:
chain.invoke({"input": "how can langsmith help with testing?"})

"LangSmith is not a tool for testing per se; it's an integrated development environment (IDE) that provides features to write, edit, and debug Java code. However, LangSmith does include some elements that can be beneficial in the testing process:\n\n1. Code Linter: LangSmith's linter can help you identify potential issues with your code before starting the actual testing phase by checking for syntax errors, style violations, and other coding best practices. This helps prevent bugs from occurring during testing.\n\n2. Test Execution Tools: LangSmith provides tools like `langtest` or `JUnit`, which are popular test execution frameworks in Java. These tools can be used to write, run, and manage tests for your applications.\n\n3. Integrated Development Environment (IDE) Features: LangSmith's IDE features such as code navigation, view types, debugger, etc., can aid you during the testing phase by allowing you to quickly traverse through your codebase, identify issues, and fix them efficient

In [5]:
# !pip install beautifulsoup4

In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

In [6]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [10]:
# !pip install faiss-cpu

   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
    --------------------------------------- 0.3/14.5 MB 7.0 MB/s eta 0:00:03
   -- ------------------------------------- 0.8/14.5 MB 8.3 MB/s eta 0:00:02
   --- ------------------------------------ 1.4/14.5 MB 9.6 MB/s eta 0:00:02
   ---- ----------------------------------- 1.7/14.5 MB 9.1 MB/s eta 0:00:02
   ------ --------------------------------- 2.3/14.5 MB 10.6 MB/s eta 0:00:02
   ------- -------------------------------- 2.9/14.5 MB 10.7 MB/s eta 0:00:02
   --------- ------------------------------ 3.6/14.5 MB 10.9 MB/s eta 0:00:01
   ----------- ---------------------------- 4.1/14.5 MB 11.0 MB/s eta 0:00:01
   ------------- -------------------------- 5.0/14.5 MB 11.0 MB/s eta 0:00:01
   --------------- ------------------------ 5.5/14.5 MB 11.1 MB/s eta 0:00:01
   ---------------- ----------------------- 5.9/14.5 MB 10.8 MB/s eta 0:00:01
   ----------------- ---------------------- 6.5/14.5 MB 10.4 MB/s eta 0:00:0

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [8]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [9]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [10]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith helps with testing by providing a platform for LLM application development, monitoring, and testing. It supports various workflows such as prototyping, beta testing, production, and A/B testing. With features like tracing, feedback collection, run annotation, dataset creation, benchmarking, and comparison view, LangSmith enables users to test their applications in different scenarios and configurations quickly. This helps identify issues and improve the performance of the LLM applications at scale.


In [11]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [12]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main contentğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, weâ€™ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if theyâ€™re just starting their journey.Prototypingâ€‹Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing â€” and debug where it is failing â€” is incredibly important for this phase.Debuggingâ€‹When developin

In [13]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [14]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="This allows you to quickly test out different prompts and models. You can open the playground from any prompt or model run in your trace.\nEvery playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testingâ€‹Beta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, itâ€™s important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly itâ€™s breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.Capturing Feedbackâ€‹When launching your application to 

In [3]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="sentence-transformers/all-MiniLM-L6-v2",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 10},
)

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

c:\Users\Dell\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
hf.invoke("Give me the definition of an algorithm")

'hing goodman gloves wings ingredients > member officers wings insurance'